In [626]:
import cv2 as cv
from enum import Enum
import numpy as np
import builtins
import collections

In [627]:
class Direction(Enum):
    VERTICAL = 0
    HORIZONTAL = 1

computeErrorSurface:
    img1 is always on top or on left of img2.

In [628]:
def computeErrorSurface(img1, img2, boundarySize: int, direction: Direction):
    border1 = border2 = None
    if direction == Direction.HORIZONTAL:
        border1 = img1[-boundarySize-1:-1,:]
        border2 = img2[0:boundarySize,:]
    else:
        border1 = img1[:,-boundarySize-1:-1]
        border2 = img2[:,0:boundarySize]

    diff = np.subtract(border1, border2)

    squareDiff = np.square(diff)

    ret = np.zeros((squareDiff.shape[0], squareDiff.shape[1]),dtype=np.uint32)

    for i in range(ret.shape[0]):
        for j in range(ret.shape[1]):
            ret[i,j] = np.sum(squareDiff[i,j])

    ret = np.sqrt(ret)
    return cv.normalize(ret,None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [629]:
def verticalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[0] != errorBoundary.shape[0]):
        neighbors.append((currentPos[0] - 1, currentPos[1]))
        if(currentPos[1] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[1] != errorBoundary.shape[1] - 1):
            neighbors.append((currentPos[0] - 1, currentPos[1] + 1))
    else:
        for i in range(errorBoundary.shape[1]):
            neighbors.append((currentPos[0] - 1, i))

    return neighbors

def horizontalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[1] != errorBoundary.shape[1]):
        neighbors.append((currentPos[0], currentPos[1] - 1))
        if(currentPos[0] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[0] != errorBoundary.shape[0] - 1):
            neighbors.append((currentPos[0] + 1, currentPos[1] - 1))
    else:
        for i in range(errorBoundary.shape[0]):
            neighbors.append((i, currentPos[1] - 1))

    return neighbors

In [630]:
# function heavily inspired by Ben Alex Keen's implementation: https://benalexkeen.com/implementing-djikstras-shortest-path-algorithm-with-python/
def minimumBoundaryCutDjikstras(errorBoundary, direction: Direction):
    dirIndex = direction.value
    invDirIndex = (direction.value + 1) % 2
    initial = [0,0]
    initial[dirIndex] = errorBoundary.shape[dirIndex]
    initial = tuple(initial)
    
    neighborFunction = verticalNeighbors

    if(direction == Direction.HORIZONTAL):
        neighborFunction = horizontalNeighbors

    shortestPaths = {initial: (None, 0)}
    currentNode = initial
    visited = set()
    
    while currentNode[dirIndex] != 0:
        visited.add(currentNode)
        destinations = neighborFunction(currentNode, errorBoundary)
        weightToCurrentNode = shortestPaths[currentNode][1]

        for nextNode in destinations:
            weight = errorBoundary[nextNode[0], nextNode[1]] + weightToCurrentNode
            if nextNode not in shortestPaths:
                shortestPaths[nextNode] = (currentNode, weight)
            else:
                currentShortestWeight = shortestPaths[nextNode][1]
                if currentShortestWeight > weight:
                    shortestPaths[nextNode] = (currentNode, weight)
        
        nextDestinations = {node: shortestPaths[node] for node in shortestPaths if node not in visited}
        currentNode = builtins.min(nextDestinations, key=lambda k: nextDestinations[k][1])
    
    path = []
    while currentNode is not None:
        path.append(currentNode[invDirIndex])
        nextNode = shortestPaths[currentNode][0]
        currentNode = nextNode

    return path[:-1]

In [631]:
def show(names, imgs, scaling = 1):
    for i in range(len(names)):
        cv.imshow(names[i], cv.resize(imgs[i], (imgs[i].shape[1] * scaling, imgs[i].shape[0] * scaling)))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [632]:
def fShow(imgs, scaling):
    for i in range(len(imgs)):
        cv.imshow("test " + str(i), cv.resize(imgs[i], (imgs[i].shape[1] * scaling, imgs[i].shape[0] * scaling)))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [633]:
def applyWithMask(fullImg, patch, position, boundarySize, cutUp = None, cutLeft = None):
    size = patch.shape[0] - boundarySize
    pixelPosition = [position[0]*size, position[1]*size]
    
    patchForPasting = cv.cvtColor(patch, cv.COLOR_RGB2RGBA)
    
    patchForPasting[:,:,3] = 255
    
    if(cutUp != None):
        for i in range(patch.shape[0]):
            patchForPasting[:cutUp[i],i,3] = 0

    if(cutLeft != None):
        for i in range(patch.shape[0]):
            patchForPasting[i,:cutLeft[i],3] = 0
  
    fullImg[pixelPosition[0]:pixelPosition[0] + patch.shape[0], pixelPosition[1]:pixelPosition[1] + patch.shape[1]]  = fullImg[pixelPosition[0]:pixelPosition[0] + patch.shape[0], pixelPosition[1]:pixelPosition[1] + patch.shape[1]] * (1 - patchForPasting[:, :, 3:] / 255) + patchForPasting[:, :, :3]  * (patchForPasting[:, :, 3:] / 255)
    return fullImg

In [634]:
def computeMinimumBoundaryCut(currentImg, boundarySize, upImg = None, leftImg = None):
    errorBoundaries = [(None, None), (None, None)]

    if(upImg is not None):
        e = computeErrorSurface(upImg, currentImg, boundarySize, Direction.HORIZONTAL)
        errorBoundaries[Direction.HORIZONTAL.value] = e

    if(leftImg is not None):
        e = computeErrorSurface(leftImg, currentImg, boundarySize, Direction.VERTICAL)
        errorBoundaries[Direction.VERTICAL.value] = e

    if(errorBoundaries[0][0] is not None and errorBoundaries[1][0] is not None):
        minima = np.minimum(errorBoundaries[0][:boundarySize, :boundarySize], errorBoundaries[1][:boundarySize, :boundarySize])
        errorBoundaries[0][:boundarySize, :boundarySize] = errorBoundaries[1][:boundarySize, :boundarySize] = minima

    if(upImg is not None):
        e = errorBoundaries[Direction.HORIZONTAL.value]
        minCut = minimumBoundaryCutDjikstras(e, Direction.HORIZONTAL)
        errorBoundaries[Direction.HORIZONTAL.value] = (minCut, e)

    if(leftImg is not None):
        e = errorBoundaries[Direction.VERTICAL.value]
        minCut = minimumBoundaryCutDjikstras(e, Direction.VERTICAL)
        errorBoundaries[Direction.VERTICAL.value] = (minCut, e)

    return errorBoundaries

In [635]:
def getCutImage(errorSurface, cut, index):
    cutImage = cv.merge((errorSurface,errorSurface,errorSurface))
    for i in range(cutImage.shape[index]):
        cutImage[i, cut[i]] = [0,255,0]
    return cutImage

In [636]:
def boundaryError(canvas, patch, position, borderSize, maxValue):
    mask = np.zeros(patch.shape, dtype=np.uint8)

    if position[0] != 0:
        mask[:borderSize,:] = 255

    if position[2] != 0:
        mask[:,:borderSize] = 255
          
    maskedPatch = cv.bitwise_and(patch, mask)
    canvasSlice = canvas[position[0]:position[1], position[2]:position[3]]
    return np.sum(np.divide(np.absolute(np.subtract(maskedPatch.astype(np.float),canvasSlice.astype(np.float))),maxValue))

In [637]:
def correspondenceError(textureMap, targetMap, texturePosition, targetPosition, maxValue):
    try:
        if texturePosition[1] >= textureMap.shape[0]:
            texturePosition[1] = textureMap.shape[0] - 1

        if texturePosition[3] >= textureMap.shape[1]:
            texturePosition[3] = textureMap.shape[1] - 1
        texture = textureMap[texturePosition[0]:texturePosition[1], texturePosition[2]:texturePosition[3]]
        h = texture.shape[0]
        w = texture.shape[1]
        target = targetMap[targetPosition[0]:targetPosition[0] + h, targetPosition[2]:targetPosition[2] + w]

        return np.sum(np.divide(np.absolute(np.subtract(target.astype(np.float),texture.astype(np.float))),maxValue))
    except: return 1

In [638]:
def choosePatch(canvas, patches, positionId, cellSize, borderSize, textureMap, targetMap, patchSize, patchGridShape):
    global randomness
    bestPatch = collections.deque([(np.inf,0)], maxlen=randomness)
    targetPosition = [positionId[0]*cellSize,(positionId[0] + 1)*cellSize + borderSize, positionId[1]*cellSize,(positionId[1]+1)*cellSize + borderSize]
    maxValue = np.multiply(np.prod(patches[0].shape), 255)

    global alpha
    for i in range(len(patches)):
        row = int(i/patchGridShape[0])
        col = i%patchGridShape[1]
        
        texturePosition = [row*patchSize,(row + 1)*patchSize, col*patchSize,(col+1)*patchSize]

        error = alpha*boundaryError(canvas, patches[i], targetPosition, borderSize, maxValue)
        error += (1-alpha)*correspondenceError(textureMap, targetMap, texturePosition, targetPosition,maxValue)

        if error < bestPatch[0][0]:
            bestPatch.appendleft((error, i))

    return bestPatch[np.random.randint(len(bestPatch))][1]

In [639]:
def quiltCanvas(patches, targetGridSize, cellSize, borderSize, textureMap, targetMap, patchSize, patchGridShape, grid=None):
    canvas = np.zeros((cellSize*targetGridSize[0] + borderSize, cellSize*targetGridSize[1] + borderSize, 3), dtype=np.uint8)
    progress = 0
    progressIncrease = 100/np.prod(targetGridSize)

    for i in range(targetGridSize[0]):
        for j in range(targetGridSize[1]):
            print(str(int(progress)) + '%')
            progress += progressIncrease
            
            upCut = leftCut = upImg = leftImg = curImg = None
            patchID = None
 
            if grid is not None:
                patchID = grid[i,j]
            else:
                if(i==j==0 and targetMap is None):
                    patchID = np.random.randint(len(patches))
                else:
                    patchID = choosePatch(canvas, patches, (i,j), cellSize, borderSize, textureMap, targetMap, patchSize, patchGridShape)

            curImg = patches[patchID]

            if i != 0:
                upImg = canvas[(i-1)*cellSize:(i)*cellSize + borderSize, j*cellSize:(j+1)*cellSize + borderSize]
            if j != 0:
                leftImg = canvas[i*cellSize:(i+1)*cellSize + borderSize, (j-1)*cellSize:j*cellSize + borderSize]

            temp = computeMinimumBoundaryCut(curImg, borderSize, upImg=upImg,leftImg=leftImg)
            upCut = temp[Direction.HORIZONTAL.value][0]
            leftCut = temp[Direction.VERTICAL.value][0]
            canvas = applyWithMask(canvas, curImg, (i,j), borderSize, cutLeft = leftCut, cutUp=upCut)
    return canvas

In [640]:
textureSource = cv.imread("./Testing/Input/noise1.jpg", cv.IMREAD_COLOR)
textureSource = cv.resize(textureSource, (int(textureSource.shape[0]*2), int(textureSource.shape[1]*2)))

targetSource = cv.imread("./Testing/Input/noise2.png", cv.IMREAD_COLOR)
targetSource = cv.resize(targetSource, (int(targetSource.shape[0]*4), int(targetSource.shape[1]*4)))

textureMap =    cv.GaussianBlur(cv.cvtColor(textureSource, cv.COLOR_BGR2GRAY), (7,7), 0.5)
targetMap  =    cv.GaussianBlur(cv.cvtColor(targetSource, cv.COLOR_BGR2GRAY), (7,7), 0.5)
   
randomness = 5
alpha = 0.7
patchSize = 30
borderRatio = 4
borderSize = int(patchSize/borderRatio)
cellSize = patchSize - borderSize

targetGridSize = (int(targetSource.shape[0]/cellSize),int(targetSource.shape[1]/cellSize))
patchGridShape = (int(textureSource.shape[0]/patchSize),int(textureSource.shape[1]/patchSize))

patches = []
for i in range(patchGridShape[0]):
    for j in range(patchGridShape[1]):
        patches.append(textureSource[i*patchSize:(i+1)*patchSize, j*patchSize:(j+1)*patchSize].copy())     

canvas = quiltCanvas(patches, targetGridSize, cellSize, borderSize, textureMap, targetMap, patchSize, patchGridShape)

show(["texture", "target", "output"], [textureSource, targetSource, canvas])

cv.imwrite('./Testing/Output/noises' + '_'.join((str(randomness), str(alpha), str(patchSize), str(borderRatio))) + '.png', canvas)

0%
0%
0%
0%
0%
0%
0%
0%
0%
0%
0%
0%
1%
1%
1%
1%
1%
1%
1%
1%
1%
1%
1%
1%
2%
2%
2%
2%
2%
2%
2%
2%
2%
2%
2%
3%
3%
3%
3%
3%
3%
3%
3%
3%
3%
3%
3%
4%
4%
4%
4%
4%
4%
4%
4%
4%
4%
4%
5%
5%
5%
5%
5%
5%
5%
5%
5%
5%
5%
5%
6%
6%
6%
6%
6%
6%
6%
6%
6%
6%
6%
7%
7%
7%
7%
7%
7%
7%
7%
7%
7%
7%
7%
8%
8%
8%
8%
8%
8%
8%
8%
8%
8%
8%
8%
9%
9%
9%
9%
9%
9%
9%
9%
9%
9%
9%
10%
10%
10%
10%
10%
10%
10%
10%
10%
10%
10%
10%
11%
11%
11%
11%
11%
11%
11%
11%
11%
11%
11%
12%
12%
12%
12%
12%
12%
12%
12%
12%
12%
12%
12%
13%
13%
13%
13%
13%
13%
13%
13%
13%
13%
13%
14%
14%
14%
14%
14%
14%
14%
14%
14%
14%
14%
14%
15%
15%
15%
15%
15%
15%
15%
15%
15%
15%
15%
16%
16%
16%
16%
16%
16%
16%
16%
16%
16%
16%
16%
17%
17%
17%
17%
17%
17%
17%
17%
17%
17%
17%
17%
18%
18%
18%
18%
18%
18%
18%
18%
18%
18%
18%
19%
19%
19%
19%
19%
19%
19%
19%
19%
19%
19%
19%
20%
20%
20%
20%
20%
20%
20%
20%
20%
20%
20%
21%
21%
21%
21%
21%
21%
21%
21%
21%
21%
21%
21%
22%
22%
22%
22%
22%
22%
22%
22%
22%
22%
22%
23%
23%
23%
23%
23%
23%
23%
23%
23%
23%
23%
23%
24%


True